In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
#import nzpy

import os
from urllib.parse import urlparse

import io
import requests
from PIL import Image
import hashlib

### Function Definition

In [5]:
def get_image_urg(skus, limit):

    url_dict = pd.DataFrame(columns=['SKU', 'URL', 'Style_Img', 'Width_Img', 'Height_Img'])

    options = Options()
    options.headless = True
    for sku in skus:

        driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe', options=options)
        driver.maximize_window()
        driver.get(f"https://www.google.com/search?q=fossil+{sku}+image&rlz=1C1GCEB_enES977ES977&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjR6bCNqq_3AhU6l2oFHZltCbUQ_AUoAXoECAEQAw&biw=1365&bih=937&dpr=1&safe=active&ssui=on#imgrc=UWQfzjIHi__2UM")
        #driver.find_element_by_xpath('//*[@id="L2AGLb"]/div').click() #Accept cookies
        time.sleep(2)

        actual_images = driver.find_elements_by_css_selector('img.rg_i.Q4LuWd')[:limit]
        for actual_image in actual_images:

            temp = {'SKU':[], 'URL':[], 'Style_Img':[], 'Width_Img':[],'Height_Img':[]}

            actual_image.click()
            time.sleep(2)
            temp_image = actual_image.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img')

            if temp_image.get_attribute('src') and 'http' in temp_image.get_attribute('src'):
                temp_url = temp_image.get_attribute('src')
                style_img = temp_image.get_attribute('style')
                width_img = style_img.split(';')[0].split(':')[1]
                height_img = style_img.split(';')[1].split(':')[1]

                temp['SKU'].append(sku)
                temp['URL'].append(temp_url)
                temp['Style_Img'].append(style_img)
                temp['Width_Img'].append(width_img)
                temp['Height_Img'].append(height_img)

                print(f"SUCCESS - SKU {sku} Scraped correctly")

                temp = pd.DataFrame(temp)
                url_dict = pd.concat([url_dict, temp])

                time.sleep(3)

    return url_dict

In [3]:
def download_image(folder_path:str, url:str, sku:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')

        file_path = os.path.join(folder_path, sku + '_' + hashlib.sha1(image_content).hexdigest()[:2] + '.jpg')

        #file_path = os.path.join(folder_path, sku + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)          
        print(f"SUCCESS - saved {url} - as {file_path}")
        
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

#### Test

In [6]:
test_sku = ['FTW7026', 'FS4616']
test_sku_df = get_image_urg(test_sku, 2)
test_sku_df.reset_index(inplace=True, drop=True)

<ipython-input-5-25848674e74f>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe', options=options)
<ipython-input-5-25848674e74f>:15: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = driver.find_elements_by_css_selector('img.rg_i.Q4LuWd')[:limit]


SUCCESS - SKU FTW7026 Scraped correctly
SUCCESS - SKU FTW7026 Scraped correctly
SUCCESS - SKU FS4616 Scraped correctly
SUCCESS - SKU FS4616 Scraped correctly


In [7]:
#Test Download Image 
[download_image(folder_path='C:/Users/142845/Documents/Python Scripts/Scraping Web/Google Images/', url = test_sku_df.URL[i], sku = test_sku_df.SKU[i]) for i in test_sku_df.index]

SUCCESS - saved https://fossil.scene7.com/is/image/FossilPartners/FTW7026_main?$sfcc_fos_large$ - as C:/Users/142845/Documents/Python Scripts/Scraping Web/Google Images/FTW7026_cb.jpg
SUCCESS - saved https://fossil.scene7.com/is/image/FossilPartners/FTW7026_onwrist?$sfcc_fos_large$ - as C:/Users/142845/Documents/Python Scripts/Scraping Web/Google Images/FTW7026_fc.jpg
SUCCESS - saved https://m.media-amazon.com/images/I/91mOvfJFOKL._AC_UY550_.jpg - as C:/Users/142845/Documents/Python Scripts/Scraping Web/Google Images/FS4616_a4.jpg
SUCCESS - saved https://img01.ztat.net/article/spp-media-p1/7ad5fef41e3e3350b5f0f17ee42c4b97/c8e7791006564733a8c8271b175250cb.jpg?imwidth=762 - as C:/Users/142845/Documents/Python Scripts/Scraping Web/Google Images/FS4616_15.jpg


[None, None, None, None]

### Romper

In [37]:
skus = ['FTW7026']

url_dict = pd.DataFrame(columns=['SKU', 'URL', 'Style_Img', 'Width_Img', 'Height_Img'])

options = Options()
options.headless = True
for sku in skus:
    temp = {'SKU':[], 'URL':[], 'Style_Img':[], 'Width_Img':[],'Height_Img':[] }

    driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe', options=options)
    driver.maximize_window()
    driver.get(f"https://www.google.com/search?q=fossil+{sku}+image&rlz=1C1GCEB_enES977ES977&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjR6bCNqq_3AhU6l2oFHZltCbUQ_AUoAXoECAEQAw&biw=1365&bih=937&dpr=1&safe=active&ssui=on#imgrc=UWQfzjIHi__2UM")
    #driver.find_element_by_xpath('//*[@id="L2AGLb"]/div').click() #Accept cookies
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]').click() #Images

    # extract image urls
    actual_images = driver.find_elements_by_css_selector('img.Q4LuWd')
    #actual_images = driver.find_elements_by_xpath('//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    time.sleep(2)
    for actual_image in actual_images[:1]:
        #if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
        temp_url = actual_image.get_attribute('src')
        width_img = actual_image.get_attribute('width')
        height_img = actual_image.get_attribute('height')

        temp['SKU'].append(sku)
        temp['URL'].append(temp_url)
        temp['Width_Img'].append(width_img)
        temp['Height_Img'].append(height_img)

        print(f"SUCCESS - SKU' {sku} Scraped correctly")

        temp = pd.DataFrame(temp)
        url_dict = pd.concat([url_dict, temp])





<ipython-input-37-f554a9382bc4>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe', options=options)
<ipython-input-37-f554a9382bc4>:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]').click() #Images
<ipython-input-37-f554a9382bc4>:18: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = driver.find_elements_by_css_selector('img.Q4LuWd')


SUCCESS - SKU' FTW7026 Scraped correctly


ValueError: arrays must all be same length

In [25]:
sku = 'FTW7026'

url_dict = pd.DataFrame(columns=['SKU', 'URL', 'Width_Img', 'Height_Img'])

driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe')
driver.maximize_window()
driver.get(f"https://www.google.com/search?q=fossil+{sku}+image&rlz=1C1GCEB_enES977ES977&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjR6bCNqq_3AhU6l2oFHZltCbUQ_AUoAXoECAEQAw&biw=1365&bih=937&dpr=1&safe=active&ssui=on#imgrc=UWQfzjIHi__2UM")
#driver.find_element_by_xpath('//*[@id="L2AGLb"]/div').click() #Accept cookies
time.sleep(2)

actual_images = driver.find_elements_by_css_selector('img.rg_i.Q4LuWd')
for actual_image in actual_images[:10]:

    temp = {'SKU':[], 'URL':[], 'Width_Img':[],'Height_Img':[] }

    temp_url = actual_image.get_attribute('src')
    width_img = actual_image.get_attribute('width')
    height_img = actual_image.get_attribute('height')

    temp['SKU'].append(sku)
    temp['URL'].append(temp_url)
    temp['Width_Img'].append(width_img)
    temp['Height_Img'].append(height_img)

    temp = pd.DataFrame(temp)

    url_dict = pd.concat([url_dict, temp])


<ipython-input-25-0beb6bc3e193>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe')
<ipython-input-25-0beb6bc3e193>:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = driver.find_elements_by_css_selector('img.rg_i.Q4LuWd')


In [26]:
url_dict

,SKU,URL,Width_Img,Height_Img
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",156,180
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",156,180
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",122,200
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",156,180
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",146,182
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",156,180
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",156,180
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",156,180
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",113,200
0,FTW7026,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",105,201


In [27]:
def get_image_urg(skus):
    url_dict = pd.DataFrame(columns=['SKU', 'URL', 'Width_Img', 'Height_Img'])

    options = Options()
    options.headless = True
    for sku in skus:
        #temp = {'SKU':[], 'URL':[], 'Style_Img':[], 'Width_Img':[],'Height_Img':[] }

        driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe', options=options)
        driver.maximize_window()
        driver.get(f"https://www.google.com/search?q=fossil+{sku}+image&rlz=1C1GCEB_enES977ES977&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjR6bCNqq_3AhU6l2oFHZltCbUQ_AUoAXoECAEQAw&biw=1365&bih=937&dpr=1&safe=active&ssui=on#imgrc=UWQfzjIHi__2UM")
        #driver.find_element_by_xpath('//*[@id="L2AGLb"]/div').click() #Accept cookies
        time.sleep(2)
        
        # extract image urls    
        actual_images = driver.find_elements_by_css_selector('img.rg_i.Q4LuWd')
        for actual_image in actual_images[:10]:
            temp = {'SKU':[], 'URL':[], 'Width_Img':[],'Height_Img':[] }
        
            temp_url = actual_image.get_attribute('src')
            width_img = actual_image.get_attribute('width')
            height_img = actual_image.get_attribute('height')
            
            temp['SKU'].append(sku)
            temp['URL'].append(temp_url)
            temp['Width_Img'].append(width_img)
            temp['Height_Img'].append(height_img)
            
            print(f"SUCCESS - SKU' {sku} Scraped correctly")

            temp = pd.DataFrame(temp)
            url_dict = pd.concat([url_dict, temp])

    return url_dict

In [29]:
test_sku = ['FTW7026', 'FS4616']
#test_sku = ['FTW7026']
test_sku_df = get_image_urg(test_sku)
test_sku_df.reset_index(inplace=True, drop=True)

<ipython-input-27-e2da364f388f>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe', options=options)
<ipython-input-27-e2da364f388f>:16: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = driver.find_elements_by_css_selector('img.rg_i.Q4LuWd')


SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FTW7026 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly
SUCCESS - SKU' FS4616 Scraped correctly


In [ ]:
def download_image(folder_path:str, url:str, sku:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        #file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        file_path = os.path.join(folder_path, sku + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)          
        print(f"SUCCESS - saved {url} - as {file_path}")
        
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")